In [85]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
import datetime
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import  accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.utils.multiclass import unique_labels
import joblib
from sklearn.ensemble import RandomForestClassifier
from pdpbox.pdp import pdp_interact, pdp_interact_plot, pdp_isolate, pdp_plot
from xgboost import plot_importance
import ast

In [86]:
 master = pd.DataFrame([[0]*37],columns=['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'])

In [87]:
for path in glob('Kickstarter_2019-08-15T03_20_03_022Z/*.csv'):
    df = pd.read_csv(path,error_bad_lines=False)
    master = pd.concat([df,master])

In [88]:
master.shape

(207622, 37)

In [89]:
# profile = master.profile_report(title='Pandas Profiling Report')
# profile.to_file(output_file="output.html")

In [90]:
master = master.drop(columns = ['friends','is_backing','is_starred','permissions','photo'
                               ,'currency_symbol','urls','source_url','id','name','slug'])

In [135]:
master[['category','location']].head()

,category,location
0,"{'id': 334, 'name': 'DIY Electronics', 'slug':...","{""id"":2455920,""name"":""Mountain View"",""slug"":""m..."
1,"{'id': 284, 'name': 'Musical', 'slug': 'theate...","{""id"":44418,""name"":""London"",""slug"":""london-gb""..."
2,"{'id': 332, 'name': 'Apps', 'slug': 'technolog...","{""id"":726874,""name"":""The Hague"",""slug"":""the-ha..."
3,"{'id': 42, 'name': 'Pop', 'slug': 'music/pop',...","{""id"":2450080,""name"":""Miami Beach"",""slug"":""mia..."
4,"{'id': 321, 'name': 'Punk', 'slug': 'music/pun...","{""id"":2504633,""name"":""Tempe"",""slug"":""tempe-az""..."


In [92]:
# need to get category and sub category for this. 
master['category'].head()[0]

'{"id":334,"name":"DIY Electronics","slug":"technology/diy electronics","position":4,"parent_id":16,"color":6526716,"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/technology/diy%20electronics"}}}'

In [93]:
master['created_at'] = master['created_at'].apply(lambda x: datetime.datetime.fromtimestamp(x))
master['launched_at'] = master['launched_at'].apply(lambda x: datetime.datetime.fromtimestamp(x))
master['deadline'] = master['deadline'].apply(lambda x: datetime.datetime.fromtimestamp(x))
#master['deadline'] = master['deadline'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%c'))

In [94]:
master[['created_at','launched_at','deadline','state']].sample(10)

,created_at,launched_at,deadline,state
514,2015-02-10 13:24:45,2015-03-16 23:25:00,2015-04-15 23:25:00,successful
3160,2016-04-28 11:02:09,2016-05-30 16:07:57,2016-06-29 16:07:57,failed
1508,2013-04-08 12:36:52,2013-04-16 08:59:25,2013-05-12 14:00:00,successful
2597,2018-10-02 01:00:30,2018-10-03 00:28:23,2018-11-02 15:35:00,successful
1957,2015-11-28 08:53:55,2016-01-31 07:58:55,2016-03-01 07:58:55,successful
223,2019-08-06 00:16:23,2019-08-07 10:50:56,2019-09-11 10:50:56,live
237,2016-04-29 10:30:39,2016-07-10 23:13:20,2016-08-09 21:00:00,successful
1617,2019-01-17 13:59:58,2019-02-19 10:09:18,2019-03-16 16:25:00,successful
1474,2019-02-27 08:10:15,2019-06-19 09:03:37,2019-07-19 15:59:00,successful
2002,2015-11-18 12:46:09,2015-11-20 06:18:14,2015-12-21 06:18:14,successful


In [95]:
master['category'] = master['category'].astype(str)

In [96]:
master['category'] = master['category'].apply(json.loads)

In [97]:
type(master['category'].head()[0])

dict

In [98]:
df = pd.concat([master.drop(['category'], axis=1), master['category'].apply(pd.Series)], axis=1)

In [99]:
df.columns

Index([           'backers_count',                    'blurb',
       'converted_pledged_amount',                  'country',
                     'created_at',                  'creator',
                       'currency',   'currency_trailing_code',
               'current_currency',                 'deadline',
          'disable_communication',                  'fx_rate',
                           'goal',             'is_starrable',
                    'launched_at',                 'location',
                        'pledged',                  'profile',
                      'spotlight',               'staff_pick',
                          'state',         'state_changed_at',
                'static_usd_rate',              'usd_pledged',
                       'usd_type',                    'color',
                             'id',                     'name',
                      'parent_id',                 'position',
                           'slug',                     

In [100]:
df[['slug','name']].head()

,slug,name
0,technology/diy electronics,DIY Electronics
1,theater/musical,Musical
2,technology/apps,Apps
3,music/pop,Pop
4,music/punk,Punk


In [101]:
df=df.drop(columns = ['name','parent_id','position','id','urls','color','state_changed_at'],axis =1)

In [102]:
new = df['slug'].str.split("/", n = 1, expand = True) 
df["category"]= new[0]
df["sub-category"] = new[1]
df.drop(columns =['slug'], inplace = True)

In [103]:
df[['category','sub-category']].head()

,category,sub-category
0,technology,diy electronics
1,theater,musical
2,technology,apps
3,music,pop
4,music,punk


In [104]:
df['campaign_duration'] = df['deadline'] - df['launched_at'] 
#type(df['launched_at'].head()[0])

In [105]:
df[['campaign_duration','deadline','launched_at','state']].head()

,campaign_duration,deadline,launched_at,state
0,60 days,2018-08-25 13:49:39,2018-06-26 13:49:39,failed
1,30 days,2018-09-27 05:49:57,2018-08-28 05:49:57,failed
2,60 days,2018-09-20 02:49:52,2018-07-22 02:49:52,failed
3,60 days,2011-12-30 21:45:36,2011-10-31 21:45:36,successful
4,52 days,2017-09-29 14:31:37,2017-08-08 14:31:37,successful


In [106]:
df['campaign_launch_delay'] = df['launched_at'] - df['created_at'] 

In [107]:
df['campaign_launch_delay'].head()

0   256 days 15:35:00
1     0 days 00:46:23
2    72 days 17:39:08
3    60 days 07:13:43
4     2 days 19:47:54
Name: campaign_launch_delay, dtype: timedelta64[ns]

In [108]:
df2 = df[df['state'] != ('live')]

In [109]:
df2['state'].value_counts()

successful    118183
failed         74143
canceled        8456
suspended        632
0                  1
Name: state, dtype: int64

In [110]:
df3 = df2[df2['state'] != 0]

In [111]:
df3['state'].value_counts()

successful    118183
failed         74143
canceled        8456
suspended        632
Name: state, dtype: int64

In [112]:
df3=df3.drop(columns = ['blurb'],axis =1)

In [113]:
df3['deadline'] = pd.to_datetime(df3['deadline'], infer_datetime_format=True)
df3['launched_at'] = pd.to_datetime(df3['launched_at'], infer_datetime_format=True)
#df3['campaign_launch_delay'] = pd.to_datetime(df3['campaign_launch_delay'], infer_datetime_format=True)
df3['created_at'] = pd.to_datetime(df3['created_at'], infer_datetime_format=True)

In [114]:
df3['campaign_duration_days']= df3['campaign_duration'].dt.days

In [115]:
df3['campaign_launch_delay_days']=df3['campaign_launch_delay'].dt.days

In [116]:
df3[['campaign_launch_delay_days','campaign_launch_delay']].head()

,campaign_launch_delay_days,campaign_launch_delay
0,256,256 days 15:35:00
1,0,0 days 00:46:23
2,72,72 days 17:39:08
3,60,60 days 07:13:43
4,2,2 days 19:47:54


In [117]:
df3['campaign_launch_delay_secs']=df3['campaign_launch_delay'].dt.seconds

In [118]:
df3['launched_at_year'] = df3['launched_at'].dt.year
df3['launched_at_month'] = df3['launched_at'].dt.month
df3['launched_at_day'] = df3['launched_at'].dt.day
df3['launched_at_hour'] = df3['launched_at'].dt.hour

In [119]:
df3['deadline_year'] = df3['deadline'].dt.year
df3['deadline_month'] = df3['deadline'].dt.month
df3['deadline_day'] = df3['deadline'].dt.day
df3['deadline_hour'] = df3['deadline'].dt.hour

In [120]:
df3['created_at_year'] = df3['created_at'].dt.year
df3['created_at_month'] = df3['created_at'].dt.month
df3['created_at_day'] = df3['created_at'].dt.day
df3['created_at_hour'] = df3['created_at'].dt.hour

In [121]:
df3=df3.drop(columns = ['deadline','launched_at','campaign_launch_delay','created_at',
                       'campaign_duration',0],axis =1)

In [122]:
numerical_features = ['fx_rate','goal',
                     'static_usd_rate','launched_at_year','campaign_duration_days',
                     'launched_at_month','launched_at_day','launched_at_hour','deadline_year',
                     'deadline_month','created_at_year','created_at_month','created_at_day',
                     'created_at_hour','campaign_launch_delay_days','campaign_launch_delay_secs']
cat_features = ['country','creator','currency','current_currency','location',
               'usd_type','category','sub-category']
bool_features = ['currency_trailing_code','staff_pick']

In [123]:
df5.columns

Index(['original_country', 'creator', 'currency', 'currency_trailing_code',
       'current_currency', 'fx_rate', 'goal', 'location', 'staff_pick',
       'state', 'static_usd_rate', 'usd_type', 'category', 'sub-category',
       'campaign_duration_days', 'campaign_launch_delay_days',
       'campaign_launch_delay_secs', 'launched_at_year', 'launched_at_month',
       'launched_at_day', 'launched_at_hour', 'deadline_year',
       'deadline_month', 'deadline_day', 'deadline_hour', 'created_at_year',
       'created_at_month', 'created_at_day', 'created_at_hour'],
      dtype='object')

In [124]:
df4 = df3[df3['state'] != 'canceled']

In [125]:
df5 = df4[df4['state'] != 'suspended']

In [126]:
df5=df5.drop(columns = ['disable_communication', 'is_starrable', 'profile','spotlight',
                       'backers_count','converted_pledged_amount','pledged','usd_pledged'],axis =1)


In [127]:
df5 = df5.rename(columns = {"country": "original_country"})

In [128]:
nunique = df5.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
#df6 = X_test_encoded.drop(cols_to_drop, axis=1)

In [129]:
cols_to_drop

Index([], dtype='object')

In [130]:
df5.to_csv('../traindata.csv', index=False)